<a href="https://colab.research.google.com/github/lakatosgabor/nav_uzemanyag_havi_info/blob/main/nav_uzemanyag_havi_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

'''
DATAFRAME ADATSTRUKTÚRA
       2023.év Ólmozatlan motorbenzin ESZ-95  (Ft/l) Gázolaj (Ft/l)  Keverék (Ft/l) LPG autógáz (Ft/l) CNGautógáz(Ft/kg)
0       január                                   655            713   530                407              1991

AZ OSZLOP INDEXEK HIVATKOZÁSHOZ
['2023.év', 'Ólmozatlan motorbenzin ESZ-95\xa0 (Ft/l)', 'Gázolaj (Ft/l)', 'Keverék (Ft/l)', 'LPG autógáz (Ft/l)', 'CNGautógáz(Ft/kg)']
'''

url = "https://nav.gov.hu/print/ugyfeliranytu/uzemanyag/" + str(datetime.datetime.now().year) + "-ban-alkalmazhato-uzemanyagarak"

#number_of_month = 10 # 1: január, 2: február, 3: március ... min 1, max 12

number_of_month = datetime.datetime.now().month #aktuális hónap sorszáma az aktuális hónap adatainak lekérdezéséhez

def get_data(url, number_of_month):
    try:
        if type(number_of_month) != int:
            raise ValueError("Hibás hónap formátum. 1 és 12 közötti hónap számot adhat meg!")
            return 0

        if (number_of_month > 12 or number_of_month < 1):
            raise ValueError("A megadott hónap sorszám nem létezik. Adjon meg 1 és 12 közötti sorszámot!")
            return 0

        month_index = number_of_month - 1
        response = requests.get(url)
        response.raise_for_status()  # Dobjon kivételt, ha hiba történt a lekérdezésnél
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table")

        if table is None:
            raise ValueError("Nem található táblázat az oldalon.")
            return 0

        table_rows = table.find_all("tr")
        data = []
        for row in table_rows:
            cols = row.find_all(["th", "td"])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        if len(data) < 2:
            raise ValueError("A táblázatban nincsenek adatok.")
            return 0

        df = pd.DataFrame(data[1:], columns=data[0])[::-1].reset_index(drop=True)

        if str(datetime.datetime.now().year) + '.év' not in df.columns or 'Gázolaj (Ft/l)' not in df.columns or 'LPG autógáz (Ft/l)' not in df.columns or 'Ólmozatlan motorbenzin ESZ-95\xa0 (Ft/l)' not in df.columns:
            raise ValueError("Hiányzó oszlopok a táblázatban.")
            return 0

        if len(df[data[0][0]]) < number_of_month:
            raise ValueError("A " + str(number_of_month) + ". hónap üzemanyag árai még nem elérhetőek. Jelenleg " + str(len(df[data[0][0]])) + ". hónapig állnak rendelkezésre adatok.")
            return 0

        selected_month = df[data[0][0]][month_index]
        disel = df[data[0][2]][month_index]
        lpg_gas = df[data[0][4]][month_index]
        petrol = df[data[0][1]][month_index]

        return selected_month, disel, lpg_gas, petrol

    except requests.exceptions.HTTPError as errh:
        print("HTTP hiba történt:", errh)
        return 0
    except requests.exceptions.ConnectionError as errc:
        print("Kapcsolódási hiba történt:", errc)
        return 0
    except requests.exceptions.Timeout as errt:
        print("Időtúllépés történt:", errt)
        return 0
    except requests.exceptions.RequestException as err:
        print("Kérési hiba történt:", err)
        return 0
    except ValueError as ve:
        print(ve)
        return 0


def main():
  if (get_data(url, number_of_month) == 0):
    return

  selected_month, disel, lpg_gas, petrol = get_data(url, number_of_month)

  if selected_month is not None:
      query_time = datetime.datetime.now()
      print("Hónap: ", selected_month)
      print("Gázolaj: ", disel)
      print("LPG gáz: ", lpg_gas)
      print("Petrol: ", petrol)
      print("Lekérdezés ideje: ", query_time)
  else:
      print("Nem sikerült lekérdezni az adatokat.")


main()


Hónap:  október
Gázolaj:  661
LPG gáz:  318
Petrol:  647
Lekérdezés ideje:  2023-10-27 10:53:41.397974
